# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Incorrect Password - provide again
Connected


# Ofertata 1

In [81]:
directory_path = r'C:\Users\leonardo.mangold\PycharmProjects\promos_inteligencia_negocio\pedidos_fede\01. Enero\ofertata_1'

all_files = os.listdir(directory_path)

files = [file for file in all_files if file.endswith('.csv')]
segmento_ofertata_1 = pd.DataFrame()

for file in files:
    df_aux = pd.read_csv(f"C:\\Users\\leonardo.mangold\\PycharmProjects\\promos_inteligencia_negocio\\pedidos_fede\\01. Enero\\ofertata_1\\{file}")
    df_aux['SEGMENTO'] = file.split('.')[0]
    segmento_ofertata_1 = pd.concat([df_aux, segmento_ofertata_1], ignore_index=True)
print(segmento_ofertata_1.shape[0])
segmento_ofertata_1.head(2)

1922214


,SUB_NAME,SUBCLASE,MARCA,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR,CORREO,SEGMENTO
0,HELADOS 2 LITROS,3200010002,CRUFI,424169,C,33913931.0,CI:3931,99361891.0,NaN,Segm 5
1,HELADOS 2 LITROS,3200010002,CRUFI,359467,C,33947099.0,CI:7099,99481357.0,NaN,Segm 5


In [82]:
segmento_ofertata_1['SEGMENTO'].value_counts()

Segm 3    1442018
Segm 5     480196
Name: SEGMENTO, dtype: int64

In [83]:
# Impactados unicos por Segmento
segmento_ofertata_1.groupby('SEGMENTO')['SOCI_SOCI_ID'].nunique().reset_index()

,SEGMENTO,SOCI_SOCI_ID
0,Segm 3,203698
1,Segm 5,68557


In [104]:
segmento_ofertata_1['SOCI_SOCI_ID'].nunique()

242662

In [84]:
query = '''
SELECT
    IM.SUBCLASE,
    FV.TICKET,
    FFM.SOCI_SOCI_ID,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTA,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_COSTO_PROM_POND * FV.VNTA_UNIDADES) AS GB1
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
WHERE
    FV.TIEM_DIA_ID BETWEEN '{inicio_sql}' AND '{fin_sql}'
GROUP BY
    ALL
'''

ofertata_1_inicio = '2024-01-04'
ofertata_1_fin = '2024-01-10'
ofertata_1_label = 'Ofertata 1'

cursor.execute(query.format(
    inicio_sql = ofertata_1_inicio,
    fin_sql = ofertata_1_fin
))

ofertata_1 = cursor.fetch_pandas_all()
ofertata_1['OFERTATA'] = '1'

# Me quedo con los movimientos del segmento cuando hayan comprado la subclase que les quise impactar

ofertata_1 = ofertata_1[(ofertata_1['SUBCLASE'].isin(segmento_ofertata_1['SUBCLASE'])) & (ofertata_1['SOCI_SOCI_ID'].isin(segmento_ofertata_1['SOCI_SOCI_ID']))]

print(ofertata_1.shape[0])
ofertata_1.head(2)

94042


,SUBCLASE,TICKET,SOCI_SOCI_ID,VENTA,GB1,OFERTATA
0,3000020002,202401093181275376,206429,282.79,87.4102,1
22,7600060004,20240109495497680,288481,69.00,8.2400,1


In [85]:
ofertata_1_grouped = ofertata_1.groupby('OFERTATA').agg(
    {'VENTA': 'sum',
     'GB1': 'sum',
     'TICKET': 'nunique',
     'SOCI_SOCI_ID':'nunique'
     }).reset_index()

ofertata_1_grouped['VENTA'] = round(ofertata_1_grouped['VENTA'] / 1000000, 1)
ofertata_1_grouped['GB1'] = round(ofertata_1_grouped['GB1'] / 1000000, 1)
ofertata_1_grouped['TICKET'] = round(ofertata_1_grouped['TICKET'] / 1000, 1)
ofertata_1_grouped['SOCI_SOCI_ID'] = round(ofertata_1_grouped['SOCI_SOCI_ID'] / 1000, 1)

ofertata_1_grouped.rename({'TICKET': 'TICKETS M',
                           'SOCI_SOCI_ID':'CLIENTES M',
                           'VENTA':'VENTA MM',
                           'GB1':'GB1 MM'
                           }, axis=1, inplace=True)

ofertata_1_grouped

,OFERTATA,VENTA MM,GB1 MM,TICKETS M,CLIENTES M
0,1,12.3,3.3,73.8,50.9


# Ofertata 2

In [86]:
# En ofertata 2 no hubo segmento. Por eso: en base a los que compraron en ofertata 2, ver cuanta gente hubiera caido en los segmentos 3 y 5

## Subclases

In [87]:
query = '''
SELECT
    DISTINCT IM.SUBCLASE
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
WHERE
    LAA.ORIN IN
    (
    '1000050223', '1000052025', '1000052026', '1000050604', '1000051793', '1000050430', '1000052904', '1000055018', '1000054557', '1000054561', '1000054554', '1000115080', '1000050183', '1000057475', '1000157003', '1000300431', '1000056430', '1000056460', '1000056444', '1000047820', '1000047821', '1000293393', '1000293392', '1000293399', '1000047464', '1000047463', '1000059577', '1000099194', '1000099195', '1000099197', '1000099200', '1000025827', '1000361491', '1000041840', '1000041838', '1000041824', '1000041839', '1000037657'
    )
'''
cursor.execute(query)
subclases_ofertata_2 = cursor.fetch_pandas_all()
subclases_ofertata_2.head(2)

,SUBCLASE
0,4000030002
1,100020002


## Movimientos

In [88]:
query = '''
SELECT
    IM.SUBCLASE,
    FV.TICKET,
    FFM.SOCI_SOCI_ID,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTA,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_COSTO_PROM_POND * FV.VNTA_UNIDADES) AS GB1
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
WHERE
    FV.TIEM_DIA_ID BETWEEN '{inicio_sql}' AND '{fin_sql}'
    AND IM.SUBCLASE IN {subclases_snow}
GROUP BY
    ALL
'''

ofertata_2_inicio = '2024-01-11'
ofertata_2_fin = '2024-01-17'
ofertata_2_label = 'Ofertata 2'

cursor.execute(query.format(
    inicio_sql = ofertata_2_inicio,
    fin_sql = ofertata_2_fin,
    subclases_snow = tuple(subclases_ofertata_2['SUBCLASE'].astype(str))
))

ofertata_2 = cursor.fetch_pandas_all()
ofertata_2['OFERTATA'] = '2'

print(ofertata_2.shape[0])
ofertata_2.head(2)

312280


,SUBCLASE,TICKET,SOCI_SOCI_ID,VENTA,GB1,OFERTATA
0,1500050001,20240114961905484,219544,96.76,-42.1504,2
1,1600090003,202401142083320442,456955,167.27,27.4100,2


## Segmentos

### Segmento 3

In [89]:
# Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

In [90]:
# Parte 1. Traigo los clientes activos que tienen score 5 en las subclases
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    )

SELECT
    SOCI_SOCI_ID,
    SUBCLASE
FROM
    BIZMETRIKS.DWH.LU_SCORING_SUBCLASE
WHERE
    SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
    AND SCORE = 5
    AND SUBCLASE IN {subclases_snow}
'''

cursor.execute(query.format(subclases_snow = tuple(ofertata_2['SUBCLASE'].unique().astype(str))))
segm_3_1 = cursor.fetch_pandas_all()
segm_3_1['K'] = segm_3_1['SOCI_SOCI_ID'].astype(str) + ' - ' + segm_3_1['SUBCLASE'].astype(str)
segm_3_1.head(2)

,SOCI_SOCI_ID,SUBCLASE,K
0,125375,1600090003,125375 - 1600090003
1,125375,200050003,125375 - 200050003


In [91]:
# Parte 2. Traigo los clientes compraron las subclases
query = '''
SELECT
    DISTINCT
    FFM.SOCI_SOCI_ID,
    IM.SUBCLASE
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
WHERE
    FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND FFM.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    AND IM.SUBCLASE IN {subclases_snow}
'''

cursor.execute(query.format(subclases_snow = tuple(ofertata_2['SUBCLASE'].unique().astype(str))))
segm_3_2 = cursor.fetch_pandas_all()
segm_3_2['K'] = segm_3_2['SOCI_SOCI_ID'].astype(str) + ' - ' + segm_3_2['SUBCLASE'].astype(str)
segm_3_2.head(2)

,SOCI_SOCI_ID,SUBCLASE,K
0,156608,8900050004,156608 - 8900050004
1,191850,7900010002,191850 - 7900010002


In [92]:
# Consolido

# Me quedo con los clientes activos con score 5 en las subclases (segm_3_1) que no hayn comprado las sublcases

segm_3 = segm_3_1[~segm_3_1['K'].isin(segm_3_2['K'])]

### Segmento 5

In [93]:
# Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [94]:
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    )

SELECT
    SOCI_SOCI_ID,
    SUBCLASE
FROM
    BIZMETRIKS.DWH.LU_SCORING_SUBCLASE
WHERE
    SOCI_SOCI_ID NOT IN (SELECT * FROM ACTIVOS)
    AND SCORE = 5
    AND SUBCLASE IN {subclases_snow}
'''

cursor.execute(query.format(subclases_snow = tuple(ofertata_2['SUBCLASE'].unique().astype(str))))
segm_5 = cursor.fetch_pandas_all()
segm_5.head(2)

,SOCI_SOCI_ID,SUBCLASE
0,1703642,10000030001
1,1703642,2900020002


### Ambos

In [95]:
segmento_ofertata_2 = pd.concat([segm_3[['SOCI_SOCI_ID', 'SUBCLASE']], segm_5]).drop_duplicates()

In [106]:
segmento_ofertata_2['SOCI_SOCI_ID'].nunique()

341042

In [96]:
ofertata_2 = ofertata_2[(ofertata_2['SUBCLASE'].isin(segmento_ofertata_2['SUBCLASE'])) & (ofertata_2['SOCI_SOCI_ID'].isin(segmento_ofertata_2['SOCI_SOCI_ID']))]
ofertata_2.head(2)

,SUBCLASE,TICKET,SOCI_SOCI_ID,VENTA,GB1,OFERTATA
1,1600090003,202401142083320442,456955,167.27,27.410000,2
2,4000030002,202401147482428740,44725,101.66,33.625999,2


In [98]:
ofertata_2_grouped = ofertata_2.groupby('OFERTATA').agg(
    {'VENTA': 'sum',
     'GB1': 'sum',
     'TICKET': 'nunique',
     'SOCI_SOCI_ID':'nunique'
     }).reset_index()

ofertata_2_grouped['VENTA'] = round(ofertata_2_grouped['VENTA'] / 1000000, 1)
ofertata_2_grouped['GB1'] = round(ofertata_2_grouped['GB1'] / 1000000, 1)
ofertata_2_grouped['TICKET'] = round(ofertata_2_grouped['TICKET'] / 1000, 1)
ofertata_2_grouped['SOCI_SOCI_ID'] = round(ofertata_2_grouped['SOCI_SOCI_ID'] / 1000, 1)

ofertata_2_grouped.rename({'TICKET': 'TICKETS M',
                           'SOCI_SOCI_ID':'CLIENTES M',
                           'VENTA':'VENTA MM',
                           'GB1':'GB1 MM'
                           }, axis=1, inplace=True)

ofertata_2_grouped

,OFERTATA,VENTA MM,GB1 MM,TICKETS M,CLIENTES M
0,2,28.1,5.8,135.4,85.1


In [99]:
pd.concat([ofertata_1_grouped, ofertata_2_grouped])

,OFERTATA,VENTA MM,GB1 MM,TICKETS M,CLIENTES M
0,1,12.3,3.3,73.8,50.9
0,2,28.1,5.8,135.4,85.1


In [101]:
ofertata_1['SUBCLASE'].nunique()

13

In [102]:
ofertata_2['SUBCLASE'].nunique()

22

# Fin